## Formulação matemática do problema da Grade Horária na UFPR Campus Pontal (Timetabling)


### Objetivo

O objetivo desta modelagem é minimizar a quantidade de vezes que os professores da UFPR Campus Pontal vão ao campus.
Vamos levar em conta dados originais obtidos em contato com a universidade e restrições reais.


### Parâmetros

Para modelar, temos as variáveis: 

* **T**: conjunto de turmas (8 turmas em LCE)
* **P**: conjunto de professores (16 professores em LCE)
* **H**: conjunto de horários das aulas (1 ou 2 - primeiro ou segundo horário)
* **D**: conjunto de dias da semana que ocorrem aulas (varia de segunda a sexta)

Total de combinações: $8 * 16 * 2 * 5 = 1280$.


### Variáveis de Decisão

As variáveis $x_{p,t,d,h}$ são inteiras e binárias, que definem se o professor $p$ irá (1) ou não (0) ministrar aula para a turma $t \in T$ no dia $d \in D$ e no horário $h \in H$.


### Problema de Otimização

$$
\begin{align}
    \text{minimizar \ \ \ \ \ } & \sum_{d=1}^{5}x_{p,t,d,h} \\
    \text{sujeito a \ \ \ \ \ } & \sum_{p=1}^{16}x_{p,t,d,h} \leq 1 \\
                                & \sum_{t=1}^{8}x_{p,t,d,h} \leq 1 \\
                                & \sum_{p=1}^{16}\sum_{h=1}^{2}x_{p,t,d,h} = HT_{t,d} \\
                                & \sum_{d=1}^{5}\sum_{h=1}^{2}x_{p,t,d,h} = R_{p,t} \\
                                & \sum_{t \in T_1}\sum_{p \in P_1}x_{p,t,d,h} = 2 \\
                                & \sum_{h=1}^{}x_{ptdh} \leq 6
\end{align}
$$

onde:

- $H$:
- $T_{t,d}$:
- $R_{p,t}$:
- $T_1$: subconjunto das turmas $T$ com aulas comuns às três habilitações
- $P_1$: subconjunto dos professores $P$ que ministram aulas nas disciplinas comuns

### Explicação das Restrições

1. Cada combinação de turma, horário e dia da semana terá somente 1 professor alocado
2. Cada combinação de professor, horário e dia da semana terá somente 1 turma sendo ministrada
3. 
4.
5.

In [18]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [19]:
def constroi_lista(df:pd.DataFrame, coluna:str):
    lista = list(df[df['Curso'].isin(['LCE', 'LCEFISICA', 'LCEMATEMATICA', 'LCEQUIMICA'])][coluna].dropna().unique())
    return lista

In [90]:
grade_compilada = pd.read_csv('dados/grade_compilada.csv', sep=';')
grade_compilada_tratado = pd.read_csv('dados/grade_compilada_tratado.csv', sep=',')
hora_aula_materia = pd.read_csv('dados/hora_aula_materia.csv', sep=',')
materias_comuns = pd.read_csv('dados/materias_comuns.csv', sep=',')
professores_materias = pd.read_csv('dados/professores_materias.csv', sep=',')

In [21]:
grade_compilada_tratado.head()

,Campus,Curso,Periodo_dia,Horario,Semestre,Dia,Periodo_Aula,Tipo_Curso,Materia,Professor,Observacao,Semestre_Curso
0,MIRASOL,EAQ,MANHA,1.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,INTRODUCAO A AQUICULTURA,LAGREZE,NaN,1_EAQ
1,MIRASOL,EAQ,MANHA,2.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,INTRODUCAO A AQUICULTURA,LAGREZE,NaN,1_EAQ
2,MIRASOL,EAQ,MANHA,3.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,PROBABILIDADE,CENDON,NaN,1_EAQ
3,MIRASOL,EAQ,MANHA,4.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,PROBABILIDADE,CENDON,NaN,1_EAQ
4,MIRASOL,EAQ,MANHA,1.0,1.0,TERCA,MATUTINO,INTEGRAL MT,INTRODUCAO A QUALIDADE,SACHSIDA,NaN,1_EAQ


In [22]:
grade_compilada_tratado['Semestre'].value_counts()

5.0    138
7.0    137
3.0    120
1.0    111
9.0     86
Name: Semestre, dtype: int64

In [23]:
hora_aula_materia.head()

,Professor,Materia,Semestre_Curso,Curso,Total_Horas,Periodos_Total
0,ALEX,CALCULO DIFERENCIAL E INTEGRAL I,1_EAS,EAS,4,2.0
1,ALEX,CALCULO I,1_ECV,ECV,4,2.0
2,ALEX,GEOMETRIA E CONSTRUCOES CEM319,5_LCEMATEMATICA,LCEMATEMATICA,4,2.0
3,ARMANI,INTRODUCAO A ENGENHARIA AMBIENTAL,1_EAS,EAS,4,2.0
4,ARMANI,POLUICAO DO AR,7_EAS,EAS,3,2.0


In [24]:
hora_aula_materia[hora_aula_materia['Curso'].isin(['LCE', 'LCEFISICA', 'LCEMATEMATICA', 'LCEQUIMICA'])]['Total_Horas'].unique()

array([4, 1, 2], dtype=int64)

In [25]:
hora_aula_materia[hora_aula_materia['Curso'].isin(['LCE', 'LCEFISICA', 'LCEMATEMATICA', 'LCEQUIMICA'])]['Periodos_Total'].unique()

array([2., 1.])

In [26]:
hora_aula_materia[hora_aula_materia['Periodos_Total']<=0.5]

,Professor,Materia,Semestre_Curso,Curso,Total_Horas,Periodos_Total


In [27]:
hora_aula_materia[hora_aula_materia['Curso'].isin(['LCE', 'LCEFISICA', 'LCEMATEMATICA', 'LCEQUIMICA'])]['Materia'].unique()

array(['GEOMETRIA E CONSTRUCOES CEM319', 'CALCULO INTEGRAL LCE131',
       'MATEMATICA V CEM306', 'FISICA I PP001', 'FISICA III PP005',
       'PRATICAS I LCE115', 'PRATICAS III LCE137', 'ANALISE CEM324',
       'TEORIA DOS NUMEROS CEM322', 'FISICA MODERNA I CEM352',
       'DIVULGACAO CIENTIFICA CEM326', 'ETICA E EDUCACAO LCE134',
       'FUNDAMENTOS DA EDUCACAO LCE113',
       'OPTATIVA I TOPICOS ESPECIAIS I LCE911',
       'PSICOLOGIA DA EDUCACAO LCE114',
       'OPTATIVA II TOPICOS ESPECIAIS II LCE912',
       'FISICA EXPERIMENTAL I CEM347', 'ESTAGIO QUIMICA I CEM364',
       'PRATICA PEDAGOGICA QUIMICA II CEM362', 'QUIMICA I LCE112',
       'FISICOQUIMICA CEM365', 'DIDATICA DAS CIENCIAS CEM334',
       'EAD HISTORIA FILOSOFIA E ENSINO LCE135',
       'EAD METODOLOGIA CIENTIFICA LCE136', 'ESTAGIO CIENCIAS I CEM336',
       'HISTORIA FILOSOFIA E ENSINO LCE135',
       'METODOLOGIA CIENTIFICA LCE136', 'GEOMETRIA ANALITICA PP002',
       'MATEMATICA ELEMENTAR LCE111', 'QUIMICA ANALITI

In [28]:
materias_comuns.head()

,Professor,Campus,Materia,Semestre_Curso
0,VALDIR,MIRASOL,PRATICA PEDAGOGICA DO ENSINO CEM335,5_LCEFISICA
1,JEINNI,MIRASOL,DIDATICA DAS CIENCIAS CEM334,5_LCEFISICA
2,JEINNI,MIRASOL,ESTAGIO CIENCIAS I CEM336,5_LCEFISICA
3,ELIANE,MIRASOL,DIVULGACAO CIENTIFICA CEM326,5_LCEFISICA
4,VALDIR,MIRASOL,PRATICA PEDAGOGICA DO ENSINO CEM335,5_LCEMATEMATICA


In [29]:
professores_materias.head()

,Professor,Campus,Curso,Tipo_Curso,Materia
0,LAGREZE,MIRASOL,EAQ,INTEGRAL MT,INTRODUCAO A AQUICULTURA
1,CENDON,MIRASOL,EAQ,INTEGRAL MT,PROBABILIDADE
2,SACHSIDA,MIRASOL,EAQ,INTEGRAL MT,INTRODUCAO A QUALIDADE
3,RODOLFO,MIRASOL,EAQ,INTEGRAL MT,INTRODUCAO A QUALIDADE
4,LUCIANA,MIRASOL,EAQ,INTEGRAL MT,GEOMETRIA ANALITICA


In [30]:
professores_materias['Curso'].value_counts()

OCEANO           49
ECV              48
EAS              39
EAQ              36
LCE              20
LCEFISICA        12
LCEMATEMATICA    10
LCEQUIMICA        9
Name: Curso, dtype: int64

In [32]:
professores_lce = list(professores_materias[professores_materias['Curso'].isin(['LCE', 'LCEFISICA', 'LCEMATEMATICA', 'LCEQUIMICA'])]['Professor'].unique())
#professores_materias['Campus'] == 'MIRASOL'
professores_lce

['GUILHERME',
 'LUCIANA',
 'BATISTA',
 'SELMA',
 'ELIANE',
 'BACALHAU',
 'TALAL',
 'ROGERIO',
 'JEINNI',
 'EMIR',
 'ELIZABETE',
 'VALDIR',
 'ALEX',
 'PEDRO',
 'CASSIO',
 'BORGES',
 'HARUMI']

## Modelagem


In [71]:
import gurobipy as gp
from gurobipy import GRB


# Inicialização do modelo
model = gp.Model("Alocação_Professores")

# Define os conjuntos
professores = constroi_lista(df = grade_compilada_tratado, coluna = 'Professor') # 17 (atual) 16 (artigo)
turmas = constroi_lista(df = grade_compilada_tratado, coluna = 'Semestre_Curso') # 5 (atual) 8 (artigo)
# atualmente não estamos segmentando o 5º e 7º em qui, fis e mat e temos o 9º que não estava no artigo
dias_semana = constroi_lista(df = grade_compilada_tratado, coluna = 'Dia') # range(1, 6)
horarios = constroi_lista(df = grade_compilada_tratado, coluna = 'Periodo_dia') # range(1, 3)
# materias_comuns_pivot = materias_comuns.pivot_table(index='Professor', columns='Materia', values='aux', fill_value=0)
professores_comuns = materias_comuns['Professor'].unique().tolist()
turmas_comuns = materias_comuns['Semestre_Curso'].unique().tolist()

# Cria as variáveis de decisão
x = model.addVars(professores, turmas, dias_semana, horarios, vtype=GRB.BINARY, name="x")

# Define a função objetivo
model.setObjective(gp.quicksum(x[p,t,d,h] for p in professores for t in turmas for d in dias_semana for h in horarios), GRB.MINIMIZE)

# Restrição 2 - Cada turma tem no máximo um professor em um horário específico
for t in turmas:
    for d in dias_semana:
        for h in horarios:
            model.addConstr(gp.quicksum(x[p,t,d,h] for p in professores) <= 1)

# Restrição 3 - Cada professor tem no máximo uma aula em um horário específico
for p in professores:
    for d in dias_semana:
        for h in horarios:
            model.addConstr(gp.quicksum(x[p,t,d,h] for t in turmas) <= 1)

# Restrição 4
# # "relação de disciplinas de cada turma"
# for t in turmas:
#     for d in dias_semana:
#         model.addConstr(gp.quicksum(x[p,t,d,h] for p in professores for h in dias_semana) == 2) # HT_t,d

# Restrição 5
# "associa cada disciplina ao professora que ira lecionar cada uma delas"
for p in professores:
    for t in turmas:
        model.addConstr(gp.quicksum(x[p,t,d,h] for d in dias_semana for h in horarios) == 4) # R_p,t

# Restrição 6
# T1 ⊂ T (subconj. das turmas com aulas comuns) e P1 ⊂ P (subconj. dos professores das aulas comuns)
for p in professores_comuns:
    for t in turmas_comuns:
        model.addConstr(gp.quicksum(x[p,t,d,h] for d in dias_semana for h in horarios) == 3)

# Restrição 7
# Garante que as aulas sejam alocadas nas 6 salas de aulas disponíveis
# Isto é, a quantidade de aulas a cada horário deve ser no máximo 6, pois esse é o limite de salas
for p in professores:
    for t in turmas:
        for d in dias_semana:
            model.addConstr(gp.quicksum(x[p,t,d,h] for h in horarios) <= 6)

model.write('modelo.lp')
# Resolve o modelo
model.optimize()

# Imprime a solução
if model.Status == GRB.OPTIMAL:
    for p, t, d, h in x:
        if x[p,t,d,h].X > 0:
            print(f"Professor {p} dá aula na disciplina {d} no turno {t} e no dia {h}")
else:
    print('O modelo não foi resolvido')

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22621.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1075 rows, 1360 columns and 5530 nonzeros


Model fingerprint: 0xce2dd41e
Variable types: 0 continuous, 1360 integer (1360 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Presolve removed 680 rows and 0 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
O modelo não foi resolvido


## modelagem incluindo diciplinas

## Ajustes no código

In [91]:
# altera o index do hora_aula_materia
hora_aula_materia.set_index(['Professor','Materia','Semestre_Curso'], inplace=True) 

#Coluna auxiliar para indicar que o professor oferta essa materia nesse semestre
hora_aula_materia['Aux'] = 1

In [92]:
hora_aula_materia

Curso  \
Professor Materia                                 Semestre_Curso                   
ALEX      CALCULO DIFERENCIAL E INTEGRAL I        1_EAS                      EAS   
          CALCULO I                               1_ECV                      ECV   
          GEOMETRIA E CONSTRUCOES CEM319          5_LCEMATEMATICA  LCEMATEMATICA   
ARMANI    INTRODUCAO A ENGENHARIA AMBIENTAL       1_EAS                      EAS   
          POLUICAO DO AR                          7_EAS                      EAS   
...                                                                          ...   
VALDIR    PRATICA PEDAGOGICA DO ENSINO CEM335     5_LCEQUIMICA        LCEQUIMICA   
          PRATICA PEDAGOGICA MATEMATICA II CEM339 7_LCEMATEMATICA  LCEMATEMATICA   
VIRNEI    FISICA GERAL                            3_EAQ                      EAQ   
          FISICA II                               3_EAS                      EAS   
          FISICA II EAD                           3_EAS                      EAS   

                                                                   Total_Horas  \
Professor Materia                                 Semestre_Curso                 
ALEX      CALCULO DIFERENCIAL E INTEGRAL I        1_EAS                      4   
          CALCULO I                               1_ECV                      4   
          GEOMETRIA E CONSTRUCOES CEM319          5_LCEMATEMATICA            4   
ARMANI    INTRODUCAO A ENGENHARIA AMBIENTAL       1_EAS                      4   
          POLUICAO DO AR                          7_EAS                      3   
...                                                                        ...   
VALDIR    PRATICA PEDAGOGICA DO ENSINO CEM335     5_LCEQUIMICA               2   
          PRATICA PEDAGOGICA MATEMATICA II CEM339 7_LCEMATEMATICA            4   
VIRNEI    FISICA GERAL                            3_EAQ                      4   
          FISICA II                               3_EAS                      3   
          FISICA II EAD                           3_EAS                      1   

                                                                   Periodos_Total  \
Professor Materia                                 Semestre_Curso                    
ALEX      CALCULO DIFERENCIAL E INTEGRAL I        1_EAS                       2.0   
          CALCULO I                               1_ECV                       2.0   
          GEOMETRIA E CONSTRUCOES CEM319          5_LCEMATEMATICA             2.0   
ARMANI    INTRODUCAO A ENGENHARIA AMBIENTAL       1_EAS                       2.0   
          POLUICAO DO AR                          7_EAS                       2.0   
...                                                                           ...   
VALDIR    PRATICA PEDAGOGICA DO ENSINO CEM335     5_LCEQUIMICA                1.0   
          PRATICA PEDAGOGICA MATEMATICA II CEM339 7_LCEMATEMATICA             2.0   
VIRNEI    FISICA GERAL                            3_EAQ                       2.0   
          FISICA II                               3_EAS                       2.0   
          FISICA II EAD                           3_EAS                       1.0   

                                                                   Aux  
Professor Materia                                 Semestre_Curso        
ALEX      CALCULO DIFERENCIAL E INTEGRAL I        1_EAS              1  
          CALCULO I                               1_ECV              1  
          GEOMETRIA E CONSTRUCOES CEM319          5_LCEMATEMATICA    1  
ARMANI    INTRODUCAO A ENGENHARIA AMBIENTAL       1_EAS              1  
          POLUICAO DO AR                          7_EAS              1  
...                                                                ...  
VALDIR    PRATICA PEDAGOGICA DO ENSINO CEM335     5_LCEQUIMICA       1  
          PRATICA PEDAGOGICA MATEMATICA II CEM339 7_LCEMATEMATICA    1  
VIRNEI    FISICA GERAL                            3_EAQ              1  
      

In [113]:
def get_hora_aula_materia(professor,materia, turma, df=hora_aula_materia):
    resultado = hora_aula_materia.query("Professor == @professor and Semestre_Curso == @turma and Materia == @materia")
    if not resultado.empty:
        return resultado['Periodos_Total'].iloc[0]
    else:
        return 0

In [116]:
get_hora_aula_materia('ALEX','CALCULO DIFERENCIAL E INTEGRAL I', '1_EAS')

2.0

In [127]:
# Inicialização do modelo
model = gp.Model("Alocação_Professores")

# Define os conjuntos
professores = constroi_lista(df = grade_compilada_tratado, coluna = 'Professor') # 17 (atual) 16 (artigo)
turmas = constroi_lista(df = grade_compilada_tratado, coluna = 'Semestre_Curso') # 5 (atual) 8 (artigo)
# atualmente não estamos segmentando o 5º e 7º em qui, fis e mat e temos o 9º que não estava no artigo
dias_semana = constroi_lista(df = grade_compilada_tratado, coluna = 'Dia') # range(1, 6)
horarios = range(1, 3) #constroi_lista(df = grade_compilada_tratado, coluna = 'Periodo_dia') # 
disciplinas = constroi_lista(df = grade_compilada_tratado, coluna = 'Materia') # Variável nova

professores_comuns = materias_comuns['Professor'].unique().tolist()
turmas_comuns = materias_comuns['Semestre_Curso'].unique().tolist()
disciplinas_comuns = materias_comuns['Materia'].unique().tolist()

# Cria as variáveis de decisão
x = model.addVars(professores, disciplinas,turmas, dias_semana, horarios, vtype=GRB.BINARY, name="x")

# Define a função objetivo
model.setObjective(gp.quicksum(x[p,m,t,d,h] 
                               for p in professores 
                               for m in disciplinas 
                               for t in turmas 
                               for d in dias_semana 
                               for h in horarios), GRB.MINIMIZE)

# Restrição 2 - Cada professor tem no máximo uma aula em um horário específico
for p in professores:
    model.addConstr(gp.quicksum(x[p,m,t,d,h] 
                                for m in disciplinas 
                                for t in turmas 
                                for d in dias_semana 
                                for h in horarios 
                                if m not in disciplinas_comuns) <= 1, name=f'R2_{p}') # precisei adicionar o if para não dar conflito com 6

# Restrição 3 - Cada turma tem no máximo um professor em um horário específico
for t in turmas:        
    model.addConstr(gp.quicksum(x[p,m,t,d,h] 
                                for p in professores 
                                for m in disciplinas 
                                for d in dias_semana 
                                for h in horarios 
                                if m not in disciplinas_comuns) <= 1, name=f'R3_{t}') # precisei adicionar o if para não dar conflito com 6

# Restrição 4
# ADICIONAR QUANTOS PERIODOS CADA MATERIA PRECISA APARECER
# "relação de disciplina de cada turma"
# for m in professores:
#     for t in turmas:
#         model.addConstr(gp.quicksum(x[p,t,m,d,h] for d in dias_semana for h in horarios) == 4, name=f'R4') # HT_t,d

# Restrição 5 
# "associa cada disciplina ao professora que ira lecionar cada uma delas e quantos periodos deve aparecer"
# Dando infeasible essa restrição,  REVISAR!!!!!
for p in professores:
  for t in turmas:
    for m in disciplinas:
        model.addConstr(gp.quicksum(x[p,m,t,d,h] for d in dias_semana for h in horarios ) == get_hora_aula_materia(p,m,t), name=f'R5_{p}_{m}_{t}') # R_p,t


# Restrição 6 # Conflito com essa tbm
# # T1 ⊂ T (subconj. das turmas com aulas comuns) e P1 ⊂ P (subconj. dos professores das aulas comuns)
for p in professores_comuns:
    for m in disciplinas_comuns:
        model.addConstr(gp.quicksum(x[p,m,t,d,h] 
                                    for d in dias_semana 
                                    for h in horarios 
                                    for t in turmas_comuns) == 3, name=f'R6_{p}_{m}') 

# Restrição 7
# Garante que as aulas sejam alocadas nas 6 salas de aulas disponíveis
# Isto é, a quantidade de aulas a cada horário deve ser no máximo 6, pois esse é o limite de salas
for h in horarios:
    for d in dias_semana:    
        model.addConstr(gp.quicksum(x[p,m,t,d,h] 
                                    for p in professores 
                                    for m in disciplinas
                                    for t in turmas) <= 6, name=f'R7_{d}_{h}')

model.write('modelo.lp')


In [128]:
# Resolve o modelo
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22621.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5623 rows, 55760 columns and 212520 nonzeros
Model fingerprint: 0xc7123e0c
Variable types: 0 continuous, 55760 integer (55760 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Presolve time: 0.04s

Explored 0 nodes (0 simplex iterations) in 0.09 seconds (0.04 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [109]:
# Supondo que model.Status == GRB.OPTIMAL e x tenha os resultados
dados = []
for p, m, t, d, h in x:
    if x[p, m, t, d, h].X > 0:
        dados.append({
            "Turma": t,
            "Professor": p,
            "Disciplina": m,            
            "Dia": d,
            "Periodo": h
        })

# Cria um DataFrame a partir dos dados
df_resultado = pd.DataFrame(dados)
df_resultado = df_resultado.sort_values(by=["Turma", "Disciplina", "Dia", "Periodo"])


          Turma Professor                          Disciplina     Dia  Periodo
    5_LCEFISICA    ELIANE        DIDATICA DAS CIENCIAS CEM334  QUARTA        1
    5_LCEFISICA    VALDIR        DIDATICA DAS CIENCIAS CEM334   SEXTA        2
    5_LCEFISICA    VALDIR        DIVULGACAO CIENTIFICA CEM326  QUARTA        1
    5_LCEFISICA    ELIANE        DIVULGACAO CIENTIFICA CEM326 SEGUNDA        2
    5_LCEFISICA    ELIANE        DIVULGACAO CIENTIFICA CEM326   SEXTA        1
    5_LCEFISICA    JEINNI        DIVULGACAO CIENTIFICA CEM326   SEXTA        1
    5_LCEFISICA    VALDIR           ESTAGIO CIENCIAS I CEM336  QUARTA        1
    5_LCEFISICA    ELIANE PRATICA PEDAGOGICA DO ENSINO CEM335  QUARTA        2
    5_LCEFISICA    ELIANE PRATICA PEDAGOGICA DO ENSINO CEM335  QUINTA        1
    5_LCEFISICA    ELIANE PRATICA PEDAGOGICA DO ENSINO CEM335   TERCA        1
5_LCEMATEMATICA    ELIANE        DIDATICA DAS CIENCIAS CEM334 SEGUNDA        1
5_LCEMATEMATICA    ELIANE        DIDATICA DAS CIENCI

In [110]:
df_resultado

,Turma,Professor,Disciplina,Dia,Periodo
3,5_LCEFISICA,ELIANE,DIDATICA DAS CIENCIAS CEM334,QUARTA,1
27,5_LCEFISICA,VALDIR,DIDATICA DAS CIENCIAS CEM334,SEXTA,2
33,5_LCEFISICA,VALDIR,DIVULGACAO CIENTIFICA CEM326,QUARTA,1
9,5_LCEFISICA,ELIANE,DIVULGACAO CIENTIFICA CEM326,SEGUNDA,2
10,5_LCEFISICA,ELIANE,DIVULGACAO CIENTIFICA CEM326,SEXTA,1
21,5_LCEFISICA,JEINNI,DIVULGACAO CIENTIFICA CEM326,SEXTA,1
30,5_LCEFISICA,VALDIR,ESTAGIO CIENCIAS I CEM336,QUARTA,1
1,5_LCEFISICA,ELIANE,PRATICA PEDAGOGICA DO ENSINO CEM335,QUARTA,2
2,5_LCEFISICA,ELIANE,PRATICA PEDAGOGICA DO ENSINO CEM335,QUINTA,1
0,5_LCEFISICA,ELIANE,PRATICA PEDAGOGICA DO ENSINO CEM335,TERCA,1
